# Data Dictionary

# 0.0 Imports

In [1]:
import pandas as pd
import inflection
from src import GeneralUtils  as gu
import seaborn as sns
import plotly.express as px
from datetime import timedelta
import matplotlib.pyplot as plt
import plotly.express as px

<frozen importlib._bootstrap>:228: RuntimeWarning: scipy._lib.messagestream.MessageStream size changed, may indicate binary incompatibility. Expected 56 from C header, got 64 from PyObject


In [2]:
def segment(score):
    if score in [555, 554, 544, 545, 454, 455, 445]:
        return 'Champions'
     
    if score in [543, 444, 435, 355, 354, 345, 344, 335]: 
        return 
    'Loyal'
    if score in [553, 551, 552, 541, 542, 533, 532, 531, 452, 451, 442, 441, 431, 453, 433, 432, 423, 353, 352, 351, 342, 341, 333, 323]: 
        return 
    'Potential Loyalist'

    if score in [512, 511, 422, 421, 412, 411, 311]: 
        return 'New Customers'

    if score in [525, 524, 523, 522, 521, 515, 514, 513, 425,424, 413,414,415, 315, 314, 313]: 
        return 'Promising'

    if score in [535, 534, 443, 434, 343, 334, 325, 324]: 
        return 'Need Attention'

    if score in [331, 321, 312, 221, 213, 231, 241, 251]: 
        return 'About To Sleep'

    if score in [255, 254, 245, 244, 253, 252, 243, 242, 235, 234, 225, 224, 153, 152, 145, 143, 142, 135, 134, 133, 125, 124]: 
        return 'At Risk'

    if score in [155, 154, 144, 214,215,115, 114, 113]: 
        return 'Cannot Lose Them'

    if score in [332, 322, 233, 232, 223, 222, 132, 123, 122, 212, 211]: 
        return 'Hibernating customers'

    if score in [111, 112, 121, 131,141,151]: 
        return 'Lost customers'

# 0.1 Load Data

In [3]:
df_raw = pd.read_csv('../data/raw/Ecommerce.csv', encoding='ISO-8859-1')

In [4]:
df_raw.head()

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,Unnamed: 8
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,29-Nov-16,2.55,17850.0,United Kingdom,NaN
1,536365,71053,WHITE METAL LANTERN,6,29-Nov-16,3.39,17850.0,United Kingdom,NaN
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,29-Nov-16,2.75,17850.0,United Kingdom,NaN
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,29-Nov-16,3.39,17850.0,United Kingdom,NaN
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,29-Nov-16,3.39,17850.0,United Kingdom,NaN


![green-divider](https://user-images.githubusercontent.com/7065401/52071924-c003ad80-2562-11e9-8297-1c6595f8a7ff.png)


# 1.0. Data Description

In [5]:
df_description = df_raw.drop('Unnamed: 8', axis=1)

In [6]:
df_description.head()

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,29-Nov-16,2.55,17850.0,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,29-Nov-16,3.39,17850.0,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,29-Nov-16,2.75,17850.0,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,29-Nov-16,3.39,17850.0,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,29-Nov-16,3.39,17850.0,United Kingdom


## 1.1. Renaming Columns

In [7]:
old_cols = ['InvoiceNo', 'StockCode', 'Description', 'Quantity','InvoiceDate', 'UnitPrice', 'CustomerID', 'Country']

snakecase = lambda x: inflection.underscore(x)
new_cols = list(map(snakecase, old_cols))

df_description.columns = new_cols

In [8]:
df_description['description'] = df_description['description'].str.lower()

## 1.2. Data Dimension

In [10]:
print(f'Number of Rows: {df_description.shape[0]}')
print(f'Number of Columnss: {df_description.shape[1]}')

Number of Rows: 541909
Number of Columnss: 8


## 1.3. Data Types & Missing Values

In [11]:
gu.sum_table(df_description)

,Name,dtypes,Uniques,Missing,Missing %
0,invoice_no,object,25900,0,0.00
1,stock_code,object,4070,0,0.00
2,description,object,4206,1454,0.27
3,quantity,int64,722,0,0.00
4,invoice_date,object,305,0,0.00
5,unit_price,float64,1630,0,0.00
6,customer_id,float64,4372,135080,24.93
7,country,object,38,0,0.00


## 1.4.  Handling Missing Values

Checking if there is elements with missing c

Checando se há invoice number dos elementos com customer id ausente em outra parte do dataset, para podermos preencher os espaços faltantes.

In [13]:
invoice_no_of_missing_customer_ids = df_description.query('customer_id.isnull()').invoice_no.values
df_description.loc[df_description.invoice_no.isin(invoice_no_of_missing_customer_ids)].query('~customer_id.isnull()')

,invoice_no,stock_code,description,quantity,invoice_date,unit_price,customer_id,country


In [14]:
# description
df_description = df_description.dropna(subset='description', axis=0)

In [15]:
df_clean = df_description .dropna(subset='customer_id', axis=0)

## 1.5. Fixing Data Types

In [16]:
df_clean.invoice_date = pd.to_datetime(df_clean.invoice_date)
df_clean.customer_id = df_clean.customer_id.astype(object)

C:\Users\Bteix\AppData\Local\Temp\ipykernel_29136\3702405254.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_clean.invoice_date = pd.to_datetime(df_clean.invoice_date)
C:\Users\Bteix\AppData\Local\Temp\ipykernel_29136\3702405254.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_clean.customer_id = df_clean.customer_id.astype(object)


## 1.6. Descriptive Statistic

### 1.6.1. Numerical Variables

In [17]:
num_att = df_clean.select_dtypes(include=['float64', 'int64'])

In [18]:
num_att.agg(['min', 'max', 'ptp', 'mean', 'median', 'var', 'std', 'skew', 'kurtosis']).T.reset_index().rename(columns={'index': 'Attributes', 'ptp': 'range'})

,Attributes,min,max,range,mean,median,var,std,skew,kurtosis
0,quantity,-80995.0,80995.0,161990.0,12.061303,5.00,61848.392291,248.693370,0.182663,94317.563673
1,unit_price,0.0,38970.0,38970.0,3.460471,1.95,4804.591645,69.315162,452.219019,246924.542988


In [19]:
num_att.describe([0.01, 0.05, 0.10, 0.20, 0.80, 0.90, 0.95, 0.99])[['quantity', 'unit_price']].T

,count,mean,std,min,1%,5%,10%,20%,50%,80%,90%,95%,99%,max
quantity,406829.0,12.061303,248.693370,-80995.0,-2.00,1.00,1.00,1.00,5.00,12.00,24.00,36.0,120.0,80995.0
unit_price,406829.0,3.460471,69.315162,0.0,0.21,0.42,0.55,0.85,1.95,4.15,6.75,8.5,15.0,38970.0


sns.histplot(df_clean.quantity)

Negative quantities are impossible so, first, we will turn it into positive number and analyze outliers.

In [20]:
df_clean['quantity'] = df_clean['quantity'].apply(lambda x: -1 * x if x < 0 else x)

C:\Users\Bteix\AppData\Local\Temp\ipykernel_29136\1036948296.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_clean['quantity'] = df_clean['quantity'].apply(lambda x: -1 * x if x < 0 else x)


![green-divider](https://user-images.githubusercontent.com/7065401/52071924-c003ad80-2562-11e9-8297-1c6595f8a7ff.png)


# 2.0. Feature Engineering

In [22]:
df_feature = df_clean.copy()

In [23]:
df_feature['total_spent'] = df_feature.quantity * df_feature.unit_price


In [24]:
# Recency
diff = df_feature.invoice_date.max() - df_feature.groupby('customer_id').max()['invoice_date']
recency = pd.DataFrame(diff).reset_index().rename(columns={'invoice_date': 'recency'})

recency['recency'] = recency['recency'].dt.days

df_feature = df_feature.merge((recency), how='inner', on='customer_id')


# Frequency
frequency = df_feature.groupby('customer_id').nunique()[['invoice_no']].reset_index().rename(columns={'invoice_no': 'frequency'})
df_feature = df_feature.merge((frequency), how='inner', on='customer_id')

# Monetary
monetary = df_feature.groupby('customer_id').mean(numeric_only=True).total_spent.reset_index().rename(columns={'total_spent': 'monetary'})
df_feature = df_feature.merge((monetary), how='inner', on='customer_id')

In [25]:
df_feature.head()

,invoice_no,stock_code,description,quantity,invoice_date,unit_price,customer_id,country,total_spent,recency,frequency,monetary
0,536365,85123A,white hanging heart t-light holder,6,2016-11-29,2.55,17850.0,United Kingdom,15.30,302,35,17.608301
1,536365,71053,white metal lantern,6,2016-11-29,3.39,17850.0,United Kingdom,20.34,302,35,17.608301
2,536365,84406B,cream cupid hearts coat hanger,8,2016-11-29,2.75,17850.0,United Kingdom,22.00,302,35,17.608301
3,536365,84029G,knitted union flag hot water bottle,6,2016-11-29,3.39,17850.0,United Kingdom,20.34,302,35,17.608301
4,536365,84029E,red woolly hottie white heart.,6,2016-11-29,3.39,17850.0,United Kingdom,20.34,302,35,17.608301


![green-divider](https://user-images.githubusercontent.com/7065401/52071924-c003ad80-2562-11e9-8297-1c6595f8a7ff.png)


# 3.0. Data Filtering

In [26]:
df_rfm = df_feature.copy()[['customer_id', 'recency', 'frequency', 'monetary']]
df_rfm.head()

,customer_id,recency,frequency,monetary
0,17850.0,302,35,17.608301
1,17850.0,302,35,17.608301
2,17850.0,302,35,17.608301
3,17850.0,302,35,17.608301
4,17850.0,302,35,17.608301


In [27]:
customers_rfm = df_rfm.drop_duplicates('customer_id')

![green-divider](https://user-images.githubusercontent.com/7065401/52071924-c003ad80-2562-11e9-8297-1c6595f8a7ff.png)


# 4.0. RFM Analysis

In [28]:
import warnings
warnings.filterwarnings('ignore')

## 4.1. Recency score

In [29]:
scores = dict(zip(pd.qcut(customers_rfm.recency, q=5).unique().sort_values(), [5, 4, 3, 2, 1]))

customers_rfm['recency_score'] = pd.qcut(customers_rfm.recency, q=5)

customers_rfm['recency_score'] = customers_rfm['recency_score'].map(scores)

In [30]:
customers_rfm.sort_values('recency', ascending=True)

,customer_id,recency,frequency,monetary,recency_score
406791,12713.0,0,1,22.330263,5
398126,12985.0,0,3,16.194103,5
11488,12748.0,0,224,8.265265,5
66621,13069.0,0,27,10.966851,5
156895,13426.0,0,12,23.471321,5
...,...,...,...,...,...
35139,14142.0,373,1,14.173182,1
35315,13065.0,373,1,14.704286,1
38394,18011.0,373,1,3.671071,1
10614,17968.0,373,1,3.262941,1


## 4.2. Frequency score

In [63]:
scores = dict(zip(pd.qcut(customers_rfm.frequency, q=6, duplicates='drop').unique().sort_values(), [1, 2, 3, 4, 5]))

customers_rfm['frequency_score'] = pd.qcut(customers_rfm.frequency, q=6, duplicates='drop')

customers_rfm['frequency_score'] = customers_rfm['frequency_score'].map(scores)

In [64]:
customers_rfm.sort_values('frequency', ascending=False)

,customer_id,recency,frequency,monetary,recency_score,frequency_score,monetary_score,rfm_score
17676,14911.0,1,248,26.270964,5,5,4,544
11488,12748.0,0,224,8.265265,5,5,1,541
23690,17841.0,1,169,5.216380,5,5,1,541
35329,14606.0,1,128,4.528918,5,5,1,541
1325,15311.0,0,118,24.936355,5,5,4,544
...,...,...,...,...,...,...,...,...
356236,15263.0,106,1,13.042500,2,1,2,212
356230,14196.0,106,1,55.920000,2,1,5,215
248181,16913.0,290,1,14.180000,1,1,2,112
356038,16340.0,107,1,3.588693,2,1,1,211


## 4.1. Monetary score

In [65]:
scores = dict(zip(pd.qcut(customers_rfm.monetary, q=5).unique().sort_values(), [1, 2, 3, 4, 5]))

customers_rfm['monetary_score'] = pd.qcut(customers_rfm.monetary, q=5)

customers_rfm['monetary_score'] = customers_rfm['monetary_score'].map(scores)

In [66]:
customers_rfm['rfm_score'] = customers_rfm[['recency_score', 'frequency_score', 'monetary_score']].apply(lambda x: f"{x['recency_score']}{x['frequency_score']}{x['monetary_score']}", axis=1)

In [67]:
customers_rfm

,customer_id,recency,frequency,monetary,recency_score,frequency_score,monetary_score,rfm_score
0,17850.0,302,35,17.608301,1,5,3,153
312,13047.0,31,18,17.326429,4,5,3,453
508,12583.0,2,18,29.384143,5,5,5,555
759,13748.0,95,5,33.866071,2,4,5,245
787,15100.0,330,6,186.150000,1,4,5,145
...,...,...,...,...,...,...,...,...
406747,13436.0,1,1,16.407500,5,1,3,513
406759,15520.0,1,1,19.083333,5,1,3,513
406777,13298.0,1,1,180.000000,5,1,5,515
406779,14569.0,1,1,18.949167,5,1,3,513


![green-divider](https://user-images.githubusercontent.com/7065401/52071924-c003ad80-2562-11e9-8297-1c6595f8a7ff.png)


# 5.0. Segmentation

In [68]:
df_seg = customers_rfm.copy()[['customer_id', 'rfm_score']]


|Segment|Scores|
|--------|-------|
Champions|	555, 554, 544, 545, 454, 455, 445
Loyal|	543, 444, 435, 355, 354, 345, 344, 335
Potential Loyalist|	553, 551, 552, 541, 542, 533, 532, 531, 452, 451, 442, 441, 431, 453, 433, 432, 423, 353, 352, 351, 342, 341, 333, 323
New Customers|	512, 511, 422, 421 412, 411, 311
Promising|	525, 524, 523, 522, 521, 515, 514, 513, 425,424, 413,414,415, 315, 314, 313
Need Attention|	535, 534, 443, 434, 343, 334, 325, 324
About To Sleep|	331, 321, 312, 221, 213, 231, 241, 251
At Risk|	255, 254, 245, 244, 253, 252, 243, 242, 235, 234, 225, 224, 153, 152, 145, 143, 142, 135, 134, 133, 125, 124
Cannot Lose Them|	155, 154, 144, 214,215,115, 114, 113
Hibernating customers|	332, 322, 233, 232, 223, 222, 132, 123, 122, 212, 211
Lost customers|	111, 112, 121, 131,141,151

In [69]:
df_seg.rfm_score = df_seg.rfm_score.astype('int')
df_seg['segment'] = df_seg['rfm_score'].apply(lambda x: segment(x))

In [70]:
df_seg

,customer_id,rfm_score,segment
0,17850.0,153,At Risk
312,13047.0,453,None
508,12583.0,555,Champions
759,13748.0,245,At Risk
787,15100.0,145,At Risk
...,...,...,...
406747,13436.0,513,Promising
406759,15520.0,513,Promising
406777,13298.0,515,Promising
406779,14569.0,513,Promising


In [71]:
df_aux = df_seg.groupby('segment').count()['customer_id'].reset_index().rename(columns={'customer_id': 'number_customers'})

px.bar(df_aux, x='segment', y='number_customers', color='segment',
       labels={'number_customers': 'Number of Customers'})

In [72]:
fig = px.treemap(df_aux, path=[px.Constant("all"), 'segment', 'number_customers'], values='number_customers', color='segment')
fig.update_traces(root_color='lightgrey')
fig.update_layout(margin=dict(t=50, l=25, r=25, b=25))

![green-divider](https://user-images.githubusercontent.com/7065401/52071924-c003ad80-2562-11e9-8297-1c6595f8a7ff.png)


# 6.0. Feature Selection

![green-divider](https://user-images.githubusercontent.com/7065401/52071924-c003ad80-2562-11e9-8297-1c6595f8a7ff.png)


# 7.0. Machine Learning Modelling

![green-divider](https://user-images.githubusercontent.com/7065401/52071924-c003ad80-2562-11e9-8297-1c6595f8a7ff.png)


# 8.0. Convert Model Performance to Business Values